### Assignment 6
### Prepared By: Woon Kim
### UNI: wk2371
### Course: ACTU 5841
### Date: March 6 2022

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('data.csv')

In [3]:
df

,x1,x2,x3,x4,x5,y
0,0.189053,-0.522748,-0.413064,-2.441467,1.799707,-1.528474
1,1.144166,-0.325423,0.773807,0.281211,-0.553823,1.246495
2,0.977567,-0.310557,-0.328824,-0.792147,0.454958,-1.869208
3,-0.099198,0.545289,-0.607186,0.126828,-0.892274,-8.154580
4,0.841465,0.188035,0.330571,0.410504,-1.010758,7.608370
...,...,...,...,...,...,...
195,3.110155,-0.818143,1.255157,-0.149256,0.260780,11.115184
196,-0.546752,1.162408,-1.254861,-1.322278,-0.207827,-1.371915
197,-1.158040,0.703278,0.921895,-0.752996,0.783836,-0.347831
198,-0.662045,-0.044232,0.006761,1.876453,-0.000428,3.607191


In [4]:
from numpy.linalg import norm

def center_normalize_df (dataframe):
    
    for column in dataframe:
        dataframe[column] = dataframe[column] - np.mean(dataframe[column])
        dataframe[column] = dataframe[column] / norm(dataframe[column])
    
    return dataframe


def center_normalize_array (array):
    array = array - np.mean(array)
    array = array / norm(array)
    
    return array

In [5]:
def R2_RMSE(y,X,B, y_hat):
    
    B = B.reshape(1,-1)
    
    y_avg = y_hat / len(y_hat)
    
    
    if (y_hat.ndim == 2):
        y_avg = y_hat[:,0] / len(y_hat[:,0])
        SSR = np.sum((y-y_hat[:,0])**2)
        SST = np.sum((y-y_avg)**2)

        R2 = 1- SSR/SST
        RMSE = np.sqrt(((y_hat[:,0] - y) ** 2).mean())
        
    else:
        y_avg = y_hat / len(y_hat)
        SSR = np.sum((y-y_hat)**2)
        SST = np.sum((y-y_avg)**2)

        R2 = 1- SSR/SST
        RMSE = np.sqrt(((y_hat - y) ** 2).mean())

    return RMSE, R2

In [6]:
def k_fold_cv_lin(B):
    from sklearn.model_selection import KFold
    
    reg = LinearRegression()
    B = B.reshape(1,-1)
    
    kf_RMSE = []
    kf_r2 = []
    
    cv = KFold(n_splits=10, shuffle=True, random_state=10)
    
    for train, test in cv.split(df):
        X_train, X_test = df.iloc[train].loc[:, df.columns != 'y'], df.iloc[test].loc[:, df.columns != 'y']
        y_train, y_test = df.iloc[train].loc[:,'y'], df.iloc[test].loc[:,'y']

        X_train  = center_normalize_array(X_train)
        
        lin = reg.fit(X_train, y_train)
        y_hat = reg.predict(X_test)

        kf_RMSE.append(np.sqrt(((y_hat - y_test) ** 2).mean()))
        kf_r2.append(R2_RMSE(y_test, X_test, B, y_hat)[1])
    

    kf_RMSE = (1/10) * np.sum(kf_RMSE)
    kf_r2 = (1/10) * np.sum(kf_r2)
    
    return kf_RMSE, kf_r2

In [7]:
def k_fold_cv_lasso(alpha, B):
    from sklearn.model_selection import KFold
    from sklearn.linear_model import Lasso
    
    kf_RMSE = []
    kf_r2 = []
    
    model = Lasso(alpha = alpha)

    cv = KFold(n_splits=10, shuffle=True, random_state=10)
    
    for train, test in cv.split(df):
        X_train, X_test = df.iloc[train].loc[:, df.columns != 'y'], df.iloc[test].loc[:, df.columns != 'y']
        y_train, y_test = df.iloc[train].loc[:,'y'], df.iloc[test].loc[:,'y']
        
        X_train  = center_normalize_array(X_train)
        
        lasso = model.fit(X_train, y_train)
        y_hat = model.predict(X_test)

        #kf_RMSE.append(np.sqrt(((y_hat - y_test) ** 2).mean()))
        kf_RMSE.append(R2_RMSE(y_test, X_test, B, y_hat)[0])
        kf_r2.append(R2_RMSE(y_test, X_test, B, y_hat)[1])
    

    kf_RMSE = (1/10) * np.sum(kf_RMSE)
    kf_r2 = (1/10) * np.sum(kf_r2)
    
    return kf_RMSE, kf_r2

In [8]:
#null model
from sklearn.linear_model import LinearRegression

df_cm = center_normalize_df(df)

X_null = np.ones((df_cm.iloc[:,0].shape[0],1))
y = df['y']
reg = LinearRegression().fit(X_null, y)
B_null = reg.coef_
B_null = np.insert(B_null, 0, reg.intercept_, axis=0)
print('The Coef are')
print(B_null)

y_hat_null = reg.predict(X_null)
r2_rmse_null = R2_RMSE(y, X_null, B_null, y_hat_null)
print('\nRMSE fit is\nRMSE = %s' % (r2_rmse_null[0]))
print('R2 fit is\nR2 = %s' % (r2_rmse_null[1]))

r2_rmse_null_cv = k_fold_cv_lin(B_null)

print('\nRMSE CV is\nRMSE = %s' % (r2_rmse_null_cv[0]))
print('R2 CV is\nR2 = %s' % (r2_rmse_null_cv[1]))

The Coef are
[-2.77555756e-18  0.00000000e+00]

RMSE fit is
RMSE = 0.07071067811865475
R2 fit is
R2 = 0.0

RMSE CV is
RMSE = 0.07664585391933734
R2 CV is
R2 = -0.5654776412806074


C:\Users\Woon\Documents\env_ds_3_10\lib\site-packages\numpy\core\fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
C:\Users\Woon\Documents\env_ds_3_10\lib\site-packages\numpy\core\fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
C:\Users\Woon\Documents\env_ds_3_10\lib\site-packages\numpy\core\fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs

In [9]:
#full model
from sklearn.linear_model import LinearRegression

df_cm = center_normalize_df(df)

X_full = df_cm[['x1', 'x2', 'x3', 'x4', 'x5']]
y = df['y']

reg2 = LinearRegression().fit(X_full, y)
B_full = reg2.coef_
B_full = np.insert(B_full, 0, reg2.intercept_, axis=0)
print('The Coef are')
print(B_full)
print('\nabs B is = ')
print(np.sum(np.abs(reg2.coef_)))

y_hat_full = reg2.predict(X_full)

r2_rmse_lin = R2_RMSE(y, X_full, B_full, y_hat_full)
print('\nRMSE fit is\nRMSE = %s' % (r2_rmse_lin[0]))
print('R2 fit is\nR2 = %s' % (r2_rmse_lin[1]))

r2_rmse_lin_cv = k_fold_cv_lin(B_full)
print('\nRMSE CV is\nRMSE = %s' % (r2_rmse_lin_cv[0]))
print('R2 CV is\nR2 = %s' % (r2_rmse_lin_cv[1]))


The Coef are
[-2.14102255e-18  2.26009703e-01  4.11413847e-01  6.18120610e-01
 -8.48310544e-02 -5.21649659e-03]

abs B is = 
1.3455917106580257

RMSE fit is
RMSE = 0.04273550300119457
R2 fit is
R2 = 0.6324079530484797

RMSE CV is
RMSE = 0.07664585391933731
R2 CV is
R2 = -0.565477641280606


C:\Users\Woon\Documents\env_ds_3_10\lib\site-packages\numpy\core\fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
C:\Users\Woon\Documents\env_ds_3_10\lib\site-packages\numpy\core\fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
C:\Users\Woon\Documents\env_ds_3_10\lib\site-packages\numpy\core\fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs

In [10]:
#lasso with lasso cv

df_cm = center_normalize_df(df)

X_lasso = df_cm[['x1', 'x2', 'x3', 'x4', 'x5']]
y = df['y']

lambdas = np.linspace(0,0.1,101)

from sklearn.model_selection import KFold
from sklearn.linear_model import LassoCV

cv = KFold(n_splits=10, shuffle=True, random_state=10)
model = LassoCV(alphas = lambdas, cv=cv)
lassocv = model.fit(X_lasso, y)

B_lasso = lassocv.coef_
B_lasso = np.insert(B_lasso, 0, lassocv.intercept_, axis=0)

print('The Coef are')
print(B_lasso)
print('\nabs B is = ')
print(np.sum(np.abs(lassocv.coef_)))
print('\nlambda best is = %f' % lassocv.alpha_)

y_hat_lasso = lassocv.predict(X_lasso)

r2_rmse_lasso = R2_RMSE(y, X_lasso, B_lasso, y_hat_lasso)
print('\nRMSE fit is\nRMSE = %s' % (r2_rmse_lasso[0]))
print('R2 fit is\nR2 = %s' % (r2_rmse_lasso[1]))


r2_rmse_lasso_cv = k_fold_cv_lasso(lassocv.alpha_, B_lasso)
print('\nRMSE CV is\nRMSE = %s' % (r2_rmse_lasso_cv[0]))
print('R2 CV is\nR2 = %s' % (r2_rmse_lasso_cv[1]))

The Coef are
[-1.11732459e-18  2.26009703e-01  4.11413847e-01  6.18120610e-01
 -8.48310544e-02 -5.21649659e-03]

abs B is = 
1.3455917106580273

lambda best is = 0.000000

RMSE fit is
RMSE = 0.04273550300119457
R2 fit is
R2 = 0.6324079530484799


C:\Users\Woon\Documents\env_ds_3_10\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Woon\Documents\env_ds_3_10\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.16406342696144283, tolerance: 8.887730997296937e-05
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Woon\Documents\env_ds_3_10\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Woon\Documents\env_ds_3_10\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is d


RMSE CV is
RMSE = 0.07664585391933726
R2 CV is
R2 = -0.5654776412806031


C:\Users\Woon\Documents\env_ds_3_10\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.655e-01, tolerance: 9.380e-05 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
C:\Users\Woon\Documents\env_ds_3_10\lib\site-packages\numpy\core\fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
C:\Users\Woon\AppData\Local\Temp\ipykernel_6224\1236838267.py:18: UserWarning: With alpha=0, this algorithm does not c

In [13]:
#Lasso with re-scaling
df_cm = center_normalize_df(df)

lambdas = np.linspace(0,0.1,101)

from sklearn.model_selection import KFold
from sklearn.linear_model import Lasso

cv = KFold(n_splits=10, shuffle=True, random_state=10)

RMSE_lambda_cv = []
RMSE_lambda = []

for i in lambdas:
    model = Lasso(alpha = i)
    
    for train, test in cv.split(df):
        X_train, X_test = df.iloc[train].loc[:, df.columns != 'y'], df.iloc[test].loc[:, df.columns != 'y']
        y_train, y_test = df.iloc[train].loc[:,'y'], df.iloc[test].loc[:,'y']
        
        X_train  = center_normalize_array(X_train)
        #print(X_train)
        
        lasso = model.fit(X_train, y_train)
        y_hat = model.predict(X_test)

        RMSE_lambda_cv.append(np.sqrt(((y_hat - y_test) ** 2).mean()))
    
    RMSE_lambda.append((1/10) * np.sum(RMSE_lambda_cv))
    RMSE_lambda_cv = []
    
df_lambda = pd.DataFrame({'lambdas':lambdas, 'RMSE': RMSE_lambda})


C:\Users\Woon\AppData\Local\Temp\ipykernel_6224\2491081572.py:24: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  lasso = model.fit(X_train, y_train)
C:\Users\Woon\Documents\env_ds_3_10\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\Woon\Documents\env_ds_3_10\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.641e-01, tolerance: 8.888e-05 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.e

In [14]:
print(df_lambda.sort_values(by=['RMSE']))

     lambdas      RMSE
0      0.000  0.043404
1      0.001  0.049597
2      0.002  0.059252
3      0.003  0.067432
73     0.073  0.070482
..       ...       ...
31     0.031  0.070482
30     0.030  0.070482
29     0.029  0.070482
27     0.027  0.070482
100    0.100  0.070482

[101 rows x 2 columns]


In [12]:
#lasso with re-scaling lamba = 0.001
df_cm = center_normalize_df(df)

X_lasso2 = df_cm[['x1', 'x2', 'x3', 'x4', 'x5']]
y = df['y']


from sklearn.linear_model import Lasso

model2 = Lasso(alpha = 0.001)
lasso2 = model2.fit(X_lasso2, y)

B_lasso2 = lasso2.coef_
B_lasso2 = np.insert(B_lasso2, 0, lasso2.intercept_, axis=0)

print('The Coef are')
print(B_lasso2)
print('\nabs B is = ')
print(np.sum(np.abs(lasso2.coef_)))

y_hat_lasso2 = lasso2.predict(X_lasso2)

r2_rmse_lasso2 = R2_RMSE(y, X_lasso2, B_lasso2, y_hat_lasso2)
print('\nRMSE fit is\nRMSE = %s' % (r2_rmse_lasso2[0]))
print('R2 fit is\nR2 = %s' % (r2_rmse_lasso2[1]))

r2_rmse_lasso2_cv = k_fold_cv_lasso(0.001, B_lasso2)
print('\nRMSE CV is\nRMSE = %s' % (r2_rmse_lasso2_cv[0]))
print('R2 CV is\nR2 = %s' % (r2_rmse_lasso2_cv[1]))

The Coef are
[-5.79745201e-19  9.66404528e-03  2.18572134e-01  4.32674031e-01
 -0.00000000e+00 -0.00000000e+00]

abs B is = 
0.6609102105491518

RMSE fit is
RMSE = 0.04945804221088604
R2 fit is
R2 = 0.5089242325087437


C:\Users\Woon\Documents\env_ds_3_10\lib\site-packages\numpy\core\fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
C:\Users\Woon\Documents\env_ds_3_10\lib\site-packages\numpy\core\fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
C:\Users\Woon\Documents\env_ds_3_10\lib\site-packages\numpy\core\fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs


RMSE CV is
RMSE = 0.054691067266911836
R2 CV is
R2 = 0.3683023512645963


C:\Users\Woon\Documents\env_ds_3_10\lib\site-packages\numpy\core\fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
C:\Users\Woon\Documents\env_ds_3_10\lib\site-packages\numpy\core\fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [13]:
#Lasso without re-scaling
df_cm = center_normalize_df(df)

lambdas = np.linspace(0,0.1,101)

from sklearn.model_selection import KFold
from sklearn import linear_model
from sklearn.linear_model import Lasso

cv = KFold(n_splits=10, shuffle=True, random_state=10)

RMSE_lambda_cv = []
RMSE_lambda = []

for i in lambdas:
    model = Lasso(alpha = i)
    
    for train, test in cv.split(df):
        X_train, X_test = df.iloc[train].loc[:, df.columns != 'y'], df.iloc[test].loc[:, df.columns != 'y']
        y_train, y_test = df.iloc[train].loc[:,'y'], df.iloc[test].loc[:,'y']
        
        #X_train  = center_normalize_array(X_train)
        lasso = model.fit(X_train, y_train)
        y_hat = model.predict(X_test)

        RMSE_lambda_cv.append(np.sqrt(((y_hat - y_test) ** 2).mean()))
    
    RMSE_lambda.append((1/10) * np.sum(RMSE_lambda_cv))
    RMSE_lambda_cv = []
    
df_lambda = pd.DataFrame({'lambdas':lambdas, 'RMSE': RMSE_lambda})


C:\Users\Woon\AppData\Local\Temp\ipykernel_21468\2884282680.py:23: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  lasso = model.fit(X_train, y_train)
C:\Users\Woon\Documents\env_ds_3_10\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\Woon\Documents\env_ds_3_10\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.641e-01, tolerance: 8.888e-05 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.

C:\Users\Woon\AppData\Local\Temp\ipykernel_21468\2884282680.py:23: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  lasso = model.fit(X_train, y_train)
C:\Users\Woon\Documents\env_ds_3_10\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\Woon\Documents\env_ds_3_10\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.712e-01, tolerance: 8.766e-05 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.

In [14]:
df_lambda.sort_values(by=['RMSE'])

,lambdas,RMSE
0,0.000,0.043624
1,0.001,0.049391
2,0.002,0.060004
3,0.003,0.068705
73,0.073,0.070482
...,...,...
31,0.031,0.070482
30,0.030,0.070482
29,0.029,0.070482
27,0.027,0.070482


In [15]:
#lasso without re-scaling lamba = 0.000
df_cm = center_normalize_df(df)

X_lasso3 = df_cm[['x1', 'x2', 'x3', 'x4', 'x5']]
y = df['y']

from sklearn.linear_model import Lasso

model3 = Lasso(alpha = 0.000)
lasso3 = model3.fit(X_lasso3, y)

B_lasso3 = lasso3.coef_
B_lasso3 = np.insert(B_lasso3, 0, lasso3.intercept_, axis=0)

print('The Coef are')
print(B_lasso3)
print('\nabs B is = ')
print(np.sum(np.abs(lasso3.coef_)))

y_hat_lasso3 = lasso3.predict(X_lasso3)

r2_rmse_lasso3 = R2_RMSE(y, X_lasso3, B_lasso3, y_hat_lasso3)
print('\nRMSE fit is\nRMSE = %s' % (r2_rmse_lasso3[0]))
print('R2 fit is\nR2 = %s' % (r2_rmse_lasso3[1]))

r2_rmse_lasso3_cv = k_fold_cv_lasso(0.000, B_lasso3)
print('\nRMSE CV is\nRMSE = %s' % (r2_rmse_lasso3_cv[0]))
print('R2 CV is\nR2 = %s' % (r2_rmse_lasso3_cv[1]))

C:\Users\Woon\AppData\Local\Temp\ipykernel_21468\2340728518.py:10: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  lasso3 = model3.fit(X_lasso3, y)
C:\Users\Woon\Documents\env_ds_3_10\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\Woon\Documents\env_ds_3_10\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.826e-01, tolerance: 1.000e-04 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.ene

The Coef are
[-8.38320962e-19  2.26009703e-01  4.11413847e-01  6.18120610e-01
 -8.48310544e-02 -5.21649659e-03]

abs B is = 
1.345591710658026

RMSE fit is
RMSE = 0.04273550300119457
R2 fit is
R2 = 0.6324079530484799


C:\Users\Woon\AppData\Local\Temp\ipykernel_21468\1236838267.py:18: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  lasso = model.fit(X_train, y_train)
C:\Users\Woon\Documents\env_ds_3_10\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\Woon\Documents\env_ds_3_10\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.529e-01, tolerance: 8.395e-05 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.


RMSE CV is
RMSE = 0.07664585391933726
R2 CV is
R2 = -0.5654776412806034


C:\Users\Woon\AppData\Local\Temp\ipykernel_21468\1236838267.py:18: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  lasso = model.fit(X_train, y_train)
C:\Users\Woon\Documents\env_ds_3_10\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\Woon\Documents\env_ds_3_10\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.656e-01, tolerance: 8.992e-05 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.